In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from pathlib import Path
import os.path
import glob
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
import os
from keras.utils import np_utils
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, VGG19, ResNet50V2, DenseNet169, EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3
from tensorflow.keras.models import Model
from tensorflow import keras
from keras.layers import TimeDistributed,Input
import scipy.stats
from sklearn.metrics import mean_absolute_error as MAE
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout, GlobalAveragePooling2D
%matplotlib inline

In [2]:
#installing the required version of xgboost
!pip install xgboost==0.90
import xgboost as xg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.5
    Uninstalling xgboost-1.7.5:
      Successfully uninstalled xgboost-1.7.5


In [3]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#unzipping the images of testing set
!unzip gdrive/My\Drive/UBFC_ROI.zip

Streaming output truncated to the last 5000 lines.
  inflating: UBFC_ROI/2/forehead82.jpg  
  inflating: UBFC_ROI/2/rightcheek73.jpg  
  inflating: UBFC_ROI/2/left_cheek72.jpg  
  inflating: UBFC_ROI/2/forehead68.jpg  
  inflating: UBFC_ROI/2/forehead71.jpg  
  inflating: UBFC_ROI/2/rightcheek71.jpg  
  inflating: UBFC_ROI/2/rightcheek68.jpg  
  inflating: UBFC_ROI/2/face68.jpg   
  inflating: UBFC_ROI/2/rightcheek66.jpg  
  inflating: UBFC_ROI/2/face63.jpg   
  inflating: UBFC_ROI/2/forehead63.jpg  
  inflating: UBFC_ROI/2/face59.jpg   
  inflating: UBFC_ROI/2/face60.jpg   
  inflating: UBFC_ROI/2/left_cheek47.jpg  
  inflating: UBFC_ROI/2/left_cheek44.jpg  
  inflating: UBFC_ROI/2/rightcheek39.jpg  
  inflating: UBFC_ROI/2/face30.jpg   
  inflating: UBFC_ROI/2/forehead27.jpg  
  inflating: UBFC_ROI/2/forehead51.jpg  
  inflating: UBFC_ROI/2/rightcheek31.jpg  
  inflating: UBFC_ROI/2/face28.jpg   
  inflating: UBFC_ROI/2/face27.jpg   
  inflating: UBFC_ROI/2/forehead21.jpg  
  inflati

In [5]:
#preprocessing the images
def load_dataset(df, m):
    paths = df['Filepath']
    images_dataset = []
    for i in range(len(paths)):
        root = paths[i]
        img = Image.open(root)
        if img.mode == 'RGBA':
            alpha = img.split()[3]
            bgmask = alpha.point(lambda x: 255-x)
            img = img.convert('RGB')
            img.paste((255,255,255), None, bgmask)
        img = img.resize((m, m), Image.ANTIALIAS)
        img = np.array(img)
        means = img.mean(axis=(0,1), dtype='float32')
        stds = img.std(axis=(0,1), dtype='float32')
        img = (img - means) / stds            
        images_dataset.append(img)
    return images_dataset

In [6]:
#analyzing the results
def corr(pred,y_test):
  m = [80, 83, 73, 42, 83, 84, 82, 117]
  pred = pred.reshape(-1,)
  y_test = y_test.reshape(-1,)
  tests = []
  preds = []
  for i in range(len(m)):
      if i == 0:
          tests.append(min(y_test[0:m[i]]))
          preds.append(min(pred[0:m[i]]))
      else:
          tests.append(min(y_test[np.sum(m[:i]):np.sum(m[:i+1])]))
          preds.append(min(pred[np.sum(m[:i]):np.sum(m[:i+1])]))

  tests = np.array(tests).reshape(-1,)
  preds = np.array(preds).reshape(-1,)
  return tests, preds

In [7]:
#uploading the images of required ROI for testing
images_test_r =  load_dataset(df = pd.read_csv('/content/gdrive/MyDrive/XG_Models/UBFC/Forehead.csv'), m = 32) #instead of Right you can choose (Left, Fore, or Face)
images_test_r = np.array(images_test_r, dtype= np.uint8).reshape(-1,30,32,32,3)

In [8]:
#uploading the ground truth of the testing set
df_test = pd.read_csv('/content/gdrive/MyDrive/XG_Models/UBFC/GT.csv')
y_test = np.array(df_test['SpO2']).reshape(-1,1)

In [9]:
#constructing the model
net_1 = EfficientNetV2B1(include_top=False,input_shape =(32,32,3), weights='imagenet') #instead of EfficientNetV2B1 you can choose (VGG16, VGG19, ResNet50V2, DenseNet169, EfficientNetV2B0, EfficientNetV2B2, or EfficientNetV2B3 )
input_pic_rightcheek = Input((30,32,32,3))
input_pic_rightcheek = Input((30,32,32,3))
E2 = net_1
output2 = GlobalAveragePooling2D()(E2.output)
cnn2 = Model(inputs=E2.input, outputs=output2)
encoded_frames2 = TimeDistributed(cnn2)(input_pic_rightcheek)
y = Model(inputs=input_pic_rightcheek, outputs=encoded_frames2)
xgb_r = xg.XGBRegressor()
xgb_r.load_model("/content/gdrive/MyDrive/XG_Models/Fore/EFB1_XG.json")#instead of EFB1 you can choose (VGG16, VGG19, ResNet50V2, DenseNet169, EfB0, EfB2, or EffB3)
                                                                       #change the path (Right, Left, Fore, Face) according to the ROI used in testing set

28456008/28456008 [==============================] - 2s 0us/step
[15:03:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
#testing the model
test = y.predict(x= images_test_r)
test = test.reshape(644,-1)

21/21 [==============================] - 36s 1s/step


In [11]:
#results of MAE
pred = xgb_r.predict(test)
mae = MAE(y_test, pred)
print('MAE: ',mae)
#results of analyzing
t,p = corr(pred,y_test)
m = p - t
j = 0
k = 0
for i in range(len(m)):
  if m[i]<0:
    j +=1
  if m[i]<2:
    k +=1
print('Percenatge of error below 0: '+ str(100 *j/len(m)))
print('Percenatge of error below 2: '+ str(100* k/len(m)))

MAE:  1.1195524227545128
Percenatge of error below 0: 50.0
Percenatge of error below 2: 100.0
